In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import csv
import json
import time
import numpy as np
import modin.pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import random

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer


import boto3
import boto
from boto.s3.key import Key


# --- catboost --- 
from catboost import CatBoostClassifier

# --- sklearn and Xgboost stuff 
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier

# --- sklearn stuff 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,  auc, roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_extraction import FeatureHasher

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, Imputer, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample

# --- import custom encoder --- 
import os
import sys


In [7]:
v1_api = [
    'customer_id',
    'ip_address',
    'email_address',
 'email_age',
 'email_alias',
 'email_domain',
 'email_type',
 'email_digit_cnt',
 'email_char_cnt',
 'email_length',
 'email_alias_length',
 'email_alt_char_cnt',
 'order_id',
 'order_timestamp',
 'order_amt',
 'order_amt_usd',
 'order_ccy_code',
 'order_tax_amt',
 'order_shipping_amt',
 'order_day_of_week',
 'order_hour',
 'order_method',
 'order_shipping_cost',
 'ordhrloc',
 'ordmeth',
 'billing_id',
 'billing_name',
 'billing_address_1',
 'billing_address_2',
 'billing_city_name',
 'billing_state_name',
 'billing_state_code_2a',
 'billing_postal_code',
 'billing_country',
 'billing_country_iso_2a',
 'shipping_id',
 'shipping_name',
 'shipping_address_1',
 'shipping_address_2',
 'shipping_city_name',
 'shipping_state_name',
 'shipping_state_code_2a',
 'shipping_postal_code',
 'shipping_country',
 'shipping_country_iso_2a',
 'shipping_create_timestamp',
 'shipping_address_age',
 'shipping_latitude',
 'shipping_longitude',
 'shipping_address_type',
 'payment_instrument',
 'avs',
 'avsnew',
 'card_fullname',
 'card_avs_code',
 'card_issuer',
 'card_subtype',
 'card_issuing_bank',
 'card_brand',
 'card_is_debit',
 'card_is_prepaid',
 'card_iso_ccy_a2',
 'card_iso_ccy_a3',
 'card_cc_flag',
 'card_is_newissuer',
 'card_is_newccy',
 'card_intl',
 'card_not_bill_ccy',
 'ip_address_int',
 'is_anonymous_proxy',
 'is_satellite_provider',
 'postal_code',
 'latitude',
 'longitude',
 'accuracy_radius',
 'continent_code',
 'continent_name',
 'country_iso_code',
 'country_name',
 'subdivision_1_iso_code',
 'subdivision_1_name',
 'subdivision_2_iso_code',
 'subdivision_2_name',
 'city_name',
 'metro_code',
 'time_zone',
 'is_in_european_union',
 'leg_country',
 'leg_region',
 'leg_city',
 'leg_postalcode',
 'leg_latitude',
 'leg_longitude',
 'leg_dmacode',
 'leg_areacode',
 'isp',
 'organization',
 'autonomous_system_number',
 'autonomous_system_organization',
 'domain',
 'connection_type',
 'leg_netspeed',
'random_int',
'fraud_target',
    
]

In [12]:
def summary_stats(df):
    """ Generate summary statsitics for a panda's data frame 
        
        Args:
            df (DataFrame): panda's dataframe to create summary statisitcs for.
    
        Returns:
            DataFrame of summary statistics 
    """
    rowcnt = len(df)
    df_s1  = df.agg(['count', 'nunique', 'min', 'max']).transpose().reset_index().rename(columns={"index":"_column"})
    df_s1["null"] = (rowcnt - df_s1["count"]).astype('int64')
    df_s1["not_null"] = rowcnt - df_s1["null"]
    df_s1["null_pct"] = df_s1["null"] / rowcnt
    df_s1["nunique_pct"] = df_s1['nunique']/ rowcnt
    dt = pd.DataFrame(df.dtypes).reset_index().rename(columns={"index":"_column", 0:"_dtype"})
    df_stats = pd.merge(dt, df_s1, on='_column', how='inner').round(4)
    df_stats['nunique'] = df_stats['nunique'].astype('int64')
    df_stats['count'] = df_stats['count'].astype('int64')
 
    return df_stats

def truncate_seq(df, key, min_seq_length, max_seq_length): 
    df.sort_values(by=[key, 'order_timestamp'], ascending=True, inplace=True) 
    gp = df.groupby([key]) 
    df = df.groupby([key]).filter(lambda x: len(x) >= min_seq_length) 
    df = df.groupby([key]).filter(lambda x: len(x) <= max_seq_length) 
    return df 

def clean_dataset(df, cat_cols, num_cols, zip_cols): 
    df = df[df.isfraud.isin([0,1])] 
    df.drop_duplicates(subset=['order_id'], inplace=True) 
    df.loc[:, 'order_timestamp'] = pd.to_datetime(df.order_timestamp, format = '%Y-%m-%d %H:%M:%S')     
    df.sort_values(['order_timestamp'], ascending=True, inplace=True) 
     
    df = clean_zip(df, zip_cols) 
    df = fillnan(df, cat_cols, num_cols) 
 
    return df 
 
def clean_zip(df, zip_cols): 
    #zipcols = ['billzip', 'shippostalcode', 'leg_postalcode'] 
    for col in zip_cols: 
        df.loc[:, col] = df[col].astype(str) 
        df.loc[:, col] = df.loc[:, col].apply(lambda x:x.split('-')[0]) 
         
    return df 
 
def fillnan(df, cat_cols, num_cols): 
    df.loc[:,cat_cols] = df.loc[:,cat_cols].fillna('<UNK>').astype('str') 
    df.loc[:,num_cols] = df.loc[:,num_cols].fillna(-1) 
     
    return df 

def create_age(df,key):
    df['order_timestamp'] = pd.to_datetime(df['order_timestamp'])
    # df.groupby(key)[['order_timestamp']].shift(1) 
    
    return df

def create_recency(df,key):
    
    df.sort_values([key, 'order_timestamp'], ascending=True, inplace=True)
    #df_shift = df.groupby(key)[['order_timestamp']].shift(1) 
    df_shift = df.groupby(key)[['order_timestamp']].shift(1).rename(columns={"order_timestamp":'order_timestamp_' +key})
    df = pd.concat([df, df_shift], axis=1)
    df[key + '_recency'] = pd.DataFrame(df['order_timestamp'].astype(np.int64)// 10**9 - df['order_timestamp_'+ key].astype(np.int64) // 10**9)
    mask = df[df['order_timestamp_'+ key].isna()].index
    df[key + '_recency'].loc[mask,] = -1
    
    return df

def create_rollcount(df,key):
    df[key + '_rollcnt'] = df.groupby(key).cumcount() + 1
    return df


def create_age(df, key): 
    '''create the age (time passed since first transaction) defined by the key''' 
     
    df.sort_values(['order_timestamp'], ascending=True, inplace=True) 
    df_first = df.drop_duplicates(subset=key, keep='first') 
    df = df.merge(df_first[key + ['order_timestamp']], on=key, suffixes = ('', '_first_'+ '_'.join(key))) 
    df['age_'+ '_'.join(key)] = (df['order_timestamp']- df['order_timestamp_first_'+ '_'.join(key)])/np.timedelta64(1, 'D')  
    df['age_'+ '_'.join(key)] = df['age_'+ '_'.join(key)].fillna(-1) 
     
    return df 
     
     
def create_OTF(df, key): 
    df.loc[:, 'order_timestamp'] = pd.to_datetime(df['order_timestamp']) 
     
    df = create_recency(df, key) 
    df = create_age(df, key) 
    df = create_rollcount(df, key) 

In [8]:
#df = pd.read_csv('s3://hawksnest-transaction-model-enc/shopbop/train_000', sep='|',low_memory=False)
all_files = ['s3://hawksnest-transaction-model-enc/amazoncom/train_000', 
             's3://hawksnest-transaction-model-enc/amazoncom/eval_000',
             's3://hawksnest-transaction-model-enc/amazoncom/test_000']
#usecols
df = pd.concat((pd.read_csv(f,sep="|", low_memory=False, usecols=v1_api) for f in all_files))
df.head()

,email_address,email_age,email_alias,email_domain,email_type,email_digit_cnt,email_char_cnt,email_length,email_alias_length,email_alt_char_cnt,...,leg_longitude,leg_dmacode,leg_areacode,isp,organization,autonomous_system_number,autonomous_system_organization,domain,connection_type,leg_netspeed
0,hoag.daniel@gmail.com,0.000000,hoag.daniel,GMAIL.COM,COM,0,10,21,11,1,...,121.5318,NaN,NaN,HiNet,HiNet,3462.0,Data Communication Business Group,hinet.net,Cable/DSL,high
1,ruby2211@hotmail.com,0.000000,ruby2211,HOTMAIL.COM,COM,4,4,20,8,0,...,127.1396,NaN,NaN,LG DACOM Corporation,LG DACOM Corporation,3786.0,LG DACOM Corporation,NaN,Cable/DSL,high
2,lmk970@naver.com,111.161748,lmk970,NAVER.COM,COM,3,3,16,6,0,...,127.1523,NaN,NaN,SK Broadband,SK Broadband,9318.0,SK Broadband Co Ltd,NaN,Cable/DSL,high
3,rohit_thakare@hotmail.com,-1.000000,rohit_thakare,HOTMAIL.COM,COM,0,12,25,13,1,...,-74.3505,501.0,908.0,Verizon Fios,Verizon Fios,701.0,"MCI Communications Services, Inc. d/b/a Verizo...",verizon.net,Cable/DSL,high
4,karenjblakeman@gmail.com,-1.000000,karenjblakeman,GMAIL.COM,COM,0,14,24,14,0,...,-75.5878,504.0,302.0,Verizon Internet Services,Verizon Internet Services,701.0,"MCI Communications Services, Inc. d/b/a Verizo...",NaN,Cable/DSL,high


In [10]:
entity_list = ['email_address', 'ip_address', 'customer_id', 'billing_id', 'shipping_id',  'card_issuing_bank',]

In [25]:
def summary_stats(df):
    """ Generate summary statsitics for a panda's data frame 
        
        Args:
            df (DataFrame): panda's dataframe to create summary statisitcs for.
    
        Returns:
            DataFrame of summary statistics 
    """
    rowcnt = len(df)
    df_s1  = df.agg(['count', 'nunique', 'min', 'max']).transpose().reset_index().rename(columns={"index":"_column"})
    #df_s1["null"] = (rowcnt - df_s1["count"])
    #df_s1["not_null"] = rowcnt - df_s1["null"]
    #df_s1["null_pct"] = df_s1["null"] / rowcnt
    #df_s1["nunique_pct"] = df_s1['nunique']/ rowcnt
    #dt = pd.DataFrame(df.dtypes).reset_index().rename(columns={"index":"_column", 0:"_dtype"})
    #df_stats = pd.merge(dt, df_s1, on='_column', how='inner')
    #df_stats['nunique'] = df_stats['nunique'].astype('int64')
    #df_stats['count'] = df_stats['count'].astype('int64')
 
    return df_s1


In [21]:
df['order_timestamp'] = pd.to_datetime(df['order_timestamp'])

In [26]:
summary_stats(df)

ValueError:  Length mismatch Expected axis has 5 elements, new values have 4 elements

ValueError:  Length mismatch Expected axis has 5 elements, new values have 4 elements

In [24]:
import modin.pandas as pd